In [ ]:
!pip install fake-useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=c3e68896d9a060524c7d74535cf2fd43a343894f34e8b6d5695591a54450f765
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


In [ ]:
%%writefile crawler.py
import gzip
import os
import pickle
import queue
import random
import requests
import time

from bs4 import BeautifulSoup as bs
from fake_useragent import UserAgent
from filelock import FileLock
from hashlib import md5


agent = UserAgent().chrome

pref = "https://simple.wikipedia.org/"
special_names = {"File", "Talk", "Category", "Template", "Wikipedia", "User", "User_talk", "Wikipedia_talk", "MediaWiki", "MediaWiki_talk" "File_talk", "Template", "template_talk", "Help", "Help_talk","Category", "Category_talk", "Portal", "Portal_talk", "Draft", "Draft_talk", "Media", "TimedText", "TimedText_talk", "Module", "Module_talk", "Special"}

def is_special_name(article):
    parts = article.split(":", 1)
    return parts[0] in special_names and len(parts) > 0

def make_dirs(path):
    os.makedirs(os.path.dirname(path), exist_ok=True)

q = queue.Queue()
visited = set()
redirects = dict()

def push(s):
    if s not in visited:
        visited.add(s)
        q.put(s)

def pop():
    if not q.empty():
        return q.get().split("#", 1)[0]
    return None

def hmd5(s):
    return md5(s.encode("utf-8")).hexdigest().lower()

directory = "/content/drive/MyDrive/twiki"
pref = "https://simple.wikipedia.org/"

url = "https://simple.wikipedia.org/wiki/Special:AllPages"
cnt = 0
if not os.path.exists(directory + "/articles.pickle"):
    while True:
        response = requests.get(url, headers={'User-Aget': agent})
        while response.status_code != 200:
            time.sleep(0.1)
            response = requests.get(url, headers={'User-Aget': agent})
        parsed = bs(response.text, 'html.parser')
        lis = parsed.find_all("div", {"class", "mw-allpages-body"})[0].find_all("li")
        for li in lis:
            a = li.find("a", {"class":False, "href":True})
            if a:
                article = a.get("href").split("/")[-1]
                if article not in visited:
                    push(article)
                    articles.append(article)
        next_page = parsed.find("div",{"class", "mw-allpages-nav"}).find_all("a")[-1]
        cnt += 1
        # print(cnt)
        if "Next page" in next_page.text:
            url = pref + next_page.get("href")
        else:
            break
else:
    with open(directory + "/articles.pickle", "rb") as f:
        articles = pickle.load(f)


if not os.path.exists(directory + "/articles.pickle"):
    with open(directory + "/articles.pickle", "wb") as f:
        pickle.dump(articles, f)

if not os.path.exists(directory + "/redirecs.pickle"):
    url = "https://simple.wikipedia.org/w/index.php?title=Special:ListRedirects&limit=500&offset=0"
    while True:
        response = requests.get(url, headers={'User-Aget': agent})
        while response.status_code != 200:
            time.sleep(0.1)
            response = requests.get(url, headers={'User-Aget': agent})
        parsed = bs(response.text, 'html.parser')
        lis = parsed.find_all("div", {"class", "mw-body-content"})[0].find_all("li")
        for li in lis:
            a = li.find_all("a")
            if len(a) == 2:
                article = a[0].get("href").split("title=")[-1].split("&redirect=no")[0]
                canonical_name = a[1].get("href").split("/")[-1]
                redirects[article] = canonical_name
                push(article)
                if canonical_name not in visited:
                    push(canonical_name)
        next_page = parsed.find("a", {"class":"mw-nextlink"})
        if next_page:
            url = pref + next_page.get("href")
        else:
            break
else:
    with open(directory + "/redirecs.pickle", "rb") as f:
        redirects = pickle.load(f)

if not os.path.exists(directory + "/redirecs.pickle"):
    with open(directory + "/redirecs.pickle", "wb") as f:
        pickle.dump(redirects, f)

random.shuffle(articles)
for article in articles:
    if not is_special_name(article):
        article_hash = hmd5(article)
        dump_path = directory + "/" + article_hash[:2] + "/" + article_hash[2:3] + "/" + article_hash[3:]
        if not os.path.exists(dump_path):
            url = pref + "wiki/" + article
            response = requests.get(url, headers={'User-Agent': agent})
            tries = 0
            if response.status_code != 404:
                while response.status_code != 200:
                    if tries > 5:
                        break
                    time.sleep(0.1)
                    response = requests.get(url, headers={'User-Agent': agent})
                    tries += 1
                    
                if tries > 5:
                    continue
                make_dirs(dump_path)
                with gzip.open(dump_path + "_", "wb") as f:
                    f.write(response.text.encode("utf-8"))
                if os.path.exists(dump_path):
                    os.remove(dump_path + "_")
                else:
                    os.rename(dump_path + "_", dump_path)
    


Writing crawler.py


In [ ]:
import subprocess

In [ ]:
subprocess.Popen(["python", "crawler.py"])
subprocess.Popen(["python", "crawler.py"])
subprocess.Popen(["python", "crawler.py"])
subprocess.Popen(["python", "crawler.py"])

In [ ]:
!ps

    PID TTY          TIME CMD
      1 ?        00:00:00 docker-init
      7 ?        00:00:11 node
     18 ?        00:00:01 tail
     54 ?        00:00:11 jupyter-noteboo
     55 ?        00:00:09 dap_multiplexer
     66 ?        00:00:35 python3
     86 ?        00:00:25 python3
    123 ?        00:00:00 bash
    124 ?        00:00:00 drive
    125 ?        00:00:00 grep
    165 ?        00:24:16 drive
    238 ?        00:00:00 bash
    239 ?        00:00:00 tail
    240 ?        00:00:00 python3
    271 ?        00:08:03 python3
    272 ?        00:08:01 python3
    275 ?        00:08:02 python3
    278 ?        00:07:59 python3
    370 ?        00:07:47 python3
    371 ?        00:06:52 python3
    374 ?        00:06:52 python3
    377 ?        00:00:08 python3 <defunct>
    765 ?        00:00:00 ps
